# bus merge

In [8]:
import pandas as pd
import numpy as np
import glob
import os

## 1. TXT파일로 변환

* CSV파일이 100만줄이 넘어서 txt로 변환 후 불러와야함.

In [146]:
df = pd.read_csv("./data2/BUS_STATION_BOARDING_MONTH_202110.txt", encoding="euc-kr", names = ["사용일자","노선번호","노선명","버스정류장ARS번호","역명","승차총승객수","하차총승객수","등록일자"], skiprows=[0])
df

C:\Users\chahn\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3331: DtypeWarning: Columns (5,6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,사용일자,노선번호,노선명,버스정류장ARS번호,역명,승차총승객수,하차총승객수,등록일자
0,20211001,130,130번(우이동~길동),09003,수유역,221,163,20211004.0
1,20211001,100,100번(하계동~용산구청),11428,한성여객종점,22,1,20211004.0
2,20211001,100,100번(하계동~용산구청),11387,노원평생학습관,12,1,20211004.0
3,20211001,100,100번(하계동~용산구청),11374,서울시립북서울미술관,52,1,20211004.0
4,20211001,100,100번(하계동~용산구청),03325,서빙고역1번출구,49,51,20211004.0
...,...,...,...,...,...,...,...,...
1191676,20211031,마포11,마포11(신촌전철역~마포역),14856,굴다리시장,33,21,20211103.0
1191677,20211031,5714,5714번(광명공영차고지~이대입구),19120,영등포시장,233,346,20211103.0
1191678,20211031,마포12,마포12(신촌전철역~마포역),14855,신석초등학교,70,66,20211103.0
1191679,20211031,173,173번(월계동~연세대),14152,서울대동창회관,76,67,20211103.0


### TXT파일 문제있는 파일
* 인코딩 불가 - 19년 9,11, 20년 1 >> 다시 다운받으니 해결
* cp949 - 20년 9 >> 다시 다운받으니 해결
* 날짜 칸 밀린것 - 20년 5,6,7 >> utf-8로 열림
* 컬럼 명 오류 - 19년 12월 >> utf-8로 열림

In [63]:
#컬럼명 오류인 파일 불러오기1
df12 = pd.read_csv("./data2/2019/BUS_STATION_BOARDING_MONTH_201912_1.csv", encoding="euc-kr", names = ["사용일자","노선ID","노선번호","노선명","표준버스정류장ID","버스정류장ARS번호","역ID","역명","승차총승객수","하차총승객수","등록일자"], skiprows=[0])
df12

,사용일자,노선ID,노선번호,노선명,표준버스정류장ID,버스정류장ARS번호,역ID,역명,승차총승객수,하차총승객수,등록일자
0,20191201,11110001,100,100번(하계동~용산구청),110000327,11428,8000658,한성여객종점,13,0,20191204
1,20191201,41110060,6513,6513번(철산동~서울대),119000021,20107,9612,공군회관,109,75,20191204
2,20191201,41110060,6513,6513번(철산동~서울대),119000020,20106,72789,서울지방병무청앞,71,68,20191204
3,20191201,41110060,6513,6513번(철산동~서울대),119000006,20006,8003058,강남중학교,108,86,20191204
4,20191201,41110060,6513,6513번(철산동~서울대),119000051,20144,9542,서울공업고등학교,165,94,20191204
...,...,...,...,...,...,...,...,...,...,...,...
1194198,20191231,11110706,영등포01,영등포01(신도림역~구로디지털단지역),118900031,19542,9008652,농협.대림2동주민센터,108,86,20200103
1194199,20191231,11110374,6640B,6640B번(양천차고지~양천차고지),116900276,17202,9030450,천왕역,171,101,20200103
1194200,20191231,11110178,2112,2112번(면목동~성북동),105000182,06268,75500,장평중학교,42,41,20200103
1194201,20191231,11111166,은평10,은평10(숭실고등학교~산새마을),111000072,12160,9102928,상신초등학교,111,55,20200103


In [64]:
#컬럼명 갖게 하기 위해 필요없는컬럼 제거
df12 = df12.drop(["노선ID", "표준버스정류장ID", "역ID"], axis=1)
df12

,사용일자,노선번호,노선명,버스정류장ARS번호,역명,승차총승객수,하차총승객수,등록일자
0,20191201,100,100번(하계동~용산구청),11428,한성여객종점,13,0,20191204
1,20191201,6513,6513번(철산동~서울대),20107,공군회관,109,75,20191204
2,20191201,6513,6513번(철산동~서울대),20106,서울지방병무청앞,71,68,20191204
3,20191201,6513,6513번(철산동~서울대),20006,강남중학교,108,86,20191204
4,20191201,6513,6513번(철산동~서울대),20144,서울공업고등학교,165,94,20191204
...,...,...,...,...,...,...,...,...
1194198,20191231,영등포01,영등포01(신도림역~구로디지털단지역),19542,농협.대림2동주민센터,108,86,20200103
1194199,20191231,6640B,6640B번(양천차고지~양천차고지),17202,천왕역,171,101,20200103
1194200,20191231,2112,2112번(면목동~성북동),06268,장평중학교,42,41,20200103
1194201,20191231,은평10,은평10(숭실고등학교~산새마을),12160,상신초등학교,111,55,20200103


In [68]:
#파일 저장
df12.to_csv("BUS_STATION_BOARDING_MONTH_201912.csv")

In [67]:
#파일 확인
df12 = pd.read_csv("./BUS_STATION_BOARDING_MONTH_201912.", encoding="utf-8", names = ["사용일자","노선번호","노선명","버스정류장ARS번호","역명","승차총승객수","하차총승객수","등록일자"], skiprows=[0])
df12

,사용일자,노선번호,노선명,버스정류장ARS번호,역명,승차총승객수,하차총승객수,등록일자
0,20191201,100,100번(하계동~용산구청),11428,한성여객종점,13,0,20191204
1,20191201,6513,6513번(철산동~서울대),20107,공군회관,109,75,20191204
2,20191201,6513,6513번(철산동~서울대),20106,서울지방병무청앞,71,68,20191204
3,20191201,6513,6513번(철산동~서울대),20006,강남중학교,108,86,20191204
4,20191201,6513,6513번(철산동~서울대),20144,서울공업고등학교,165,94,20191204
...,...,...,...,...,...,...,...,...
1194198,20191231,영등포01,영등포01(신도림역~구로디지털단지역),19542,농협.대림2동주민센터,108,86,20200103
1194199,20191231,6640B,6640B번(양천차고지~양천차고지),17202,천왕역,171,101,20200103
1194200,20191231,2112,2112번(면목동~성북동),06268,장평중학교,42,41,20200103
1194201,20191231,은평10,은평10(숭실고등학교~산새마을),12160,상신초등학교,111,55,20200103


In [32]:
#컬럼명 오류인 파일 불러오기2
df7 = pd.read_csv("./data2/2020/BUS_STATION_BOARDING_MONTH_202007.txt", encoding="euc-kr", names = ["사용일자","지울것","노선번호","노선명","버스정류장ARS번호","역명","승차총승객수","하차총승객수","등록일자"], skiprows=[0])
df7

,사용일자,지울것,노선번호,노선명,버스정류장ARS번호,역명,승차총승객수,하차총승객수,등록일자
0,20200701,11110001,100,100번(하계동~용산구청),11428,한성여객종점,19,8,20200704
1,20200701,11110001,100,100번(하계동~용산구청),11373,중계역,51,9,20200704
2,20200701,11110001,100,100번(하계동~용산구청),11376,중계역2번출구,121,22,20200704
3,20200701,11110001,100,100번(하계동~용산구청),11426,하계역,579,238,20200704
4,20200701,11110001,100,100번(하계동~용산구청),08009,삼선교.한성대학교,131,259,20200704
...,...,...,...,...,...,...,...,...,...
1202638,20200731,91000034,9714,9714번(교하운정~서울역),35626,행신초등학교(중),7,21,20200803
1202639,20200731,91000034,9714,9714번(교하운정~서울역),63656,청석마을8단지,0,31,20200803
1202640,20200731,91000034,9714,9714번(교하운정~서울역),13005,모래내시장.가좌역,5,10,20200803
1202641,20200731,91000034,9714,9714번(교하운정~서울역),12008,디지털미디어시티역,25,1,20200803


In [33]:
#컬럼명 갖게 하기 위해 필요없는컬럼 제거
df7 = df7.drop(["지울것"],axis=1)
df7

,사용일자,노선번호,노선명,버스정류장ARS번호,역명,승차총승객수,하차총승객수,등록일자
0,20200701,100,100번(하계동~용산구청),11428,한성여객종점,19,8,20200704
1,20200701,100,100번(하계동~용산구청),11373,중계역,51,9,20200704
2,20200701,100,100번(하계동~용산구청),11376,중계역2번출구,121,22,20200704
3,20200701,100,100번(하계동~용산구청),11426,하계역,579,238,20200704
4,20200701,100,100번(하계동~용산구청),08009,삼선교.한성대학교,131,259,20200704
...,...,...,...,...,...,...,...,...
1202638,20200731,9714,9714번(교하운정~서울역),35626,행신초등학교(중),7,21,20200803
1202639,20200731,9714,9714번(교하운정~서울역),63656,청석마을8단지,0,31,20200803
1202640,20200731,9714,9714번(교하운정~서울역),13005,모래내시장.가좌역,5,10,20200803
1202641,20200731,9714,9714번(교하운정~서울역),12008,디지털미디어시티역,25,1,20200803


In [35]:
#파일 저장
df7.to_csv("BUS_STATION_BOARDING_MONTH_202007_1.csv")

In [37]:
df7 = pd.read_csv("./BUS_STATION_BOARDING_MONTH_202007_1.csv", encoding="utf-8")
df7

,Unnamed: 0,사용일자,노선번호,노선명,버스정류장ARS번호,역명,승차총승객수,하차총승객수,등록일자
0,0,20200701,100,100번(하계동~용산구청),11428,한성여객종점,19,8,20200704
1,1,20200701,100,100번(하계동~용산구청),11373,중계역,51,9,20200704
2,2,20200701,100,100번(하계동~용산구청),11376,중계역2번출구,121,22,20200704
3,3,20200701,100,100번(하계동~용산구청),11426,하계역,579,238,20200704
4,4,20200701,100,100번(하계동~용산구청),08009,삼선교.한성대학교,131,259,20200704
...,...,...,...,...,...,...,...,...,...
1202638,1202638,20200731,9714,9714번(교하운정~서울역),35626,행신초등학교(중),7,21,20200803
1202639,1202639,20200731,9714,9714번(교하운정~서울역),63656,청석마을8단지,0,31,20200803
1202640,1202640,20200731,9714,9714번(교하운정~서울역),13005,모래내시장.가좌역,5,10,20200803
1202641,1202641,20200731,9714,9714번(교하운정~서울역),12008,디지털미디어시티역,25,1,20200803


In [39]:
df7 = df7.drop("Unnamed: 0", axis=1)
df7

,사용일자,노선번호,노선명,버스정류장ARS번호,역명,승차총승객수,하차총승객수,등록일자
0,20200701,100,100번(하계동~용산구청),11428,한성여객종점,19,8,20200704
1,20200701,100,100번(하계동~용산구청),11373,중계역,51,9,20200704
2,20200701,100,100번(하계동~용산구청),11376,중계역2번출구,121,22,20200704
3,20200701,100,100번(하계동~용산구청),11426,하계역,579,238,20200704
4,20200701,100,100번(하계동~용산구청),08009,삼선교.한성대학교,131,259,20200704
...,...,...,...,...,...,...,...,...
1202638,20200731,9714,9714번(교하운정~서울역),35626,행신초등학교(중),7,21,20200803
1202639,20200731,9714,9714번(교하운정~서울역),63656,청석마을8단지,0,31,20200803
1202640,20200731,9714,9714번(교하운정~서울역),13005,모래내시장.가좌역,5,10,20200803
1202641,20200731,9714,9714번(교하운정~서울역),12008,디지털미디어시티역,25,1,20200803


In [41]:
#다시 csv파일로 저장
df7.to_csv("BUS_STATION_BOARDING_MONTH_202007_2.csv")

In [45]:
#컬럼명 오류인 파일 불러오기3
df6 = pd.read_csv("./data2/2020/BUS_STATION_BOARDING_MONTH_202006.txt", encoding="euc-kr", names = ["사용일자","지울것","노선번호","노선명","버스정류장ARS번호","역명","승차총승객수","하차총승객수","등록일자"], skiprows=[0])
df6

,사용일자,지울것,노선번호,노선명,버스정류장ARS번호,역명,승차총승객수,하차총승객수,등록일자
0,20200601,41110116,9709,9709번(파주시 맥금동~서울역),12023,불광역3호선.서울혁신파크,7,1,20200604
1,20200601,41110116,9709,9709번(파주시 맥금동~서울역),12026,한전성서지사.북한산푸르지오,5,5,20200604
2,20200601,41110116,9709,9709번(파주시 맥금동~서울역),12025,한전성서지사.북한산푸르지오,1,2,20200604
3,20200601,41110116,9709,9709번(파주시 맥금동~서울역),12028,녹번역,3,8,20200604
4,20200601,41110116,9709,9709번(파주시 맥금동~서울역),12027,녹번역,9,3,20200604
...,...,...,...,...,...,...,...,...,...
1162348,20200630,91000034,9714,9714번(교하운정~서울역),63657,트리플메디컬타운,90,1,20200703
1162349,20200630,91000034,9714,9714번(교하운정~서울역),63659,중앙공원,32,0,20200703
1162350,20200630,91000034,9714,9714번(교하운정~서울역),63674,책향기마을,38,1,20200703
1162351,20200630,91000034,9714,9714번(교하운정~서울역),36601,대화역,18,71,20200703


In [46]:
#컬럼명 갖게 하기 위해 필요없는컬럼 제거
df6 = df6.drop(["지울것"],axis=1)
df6

,사용일자,노선번호,노선명,버스정류장ARS번호,역명,승차총승객수,하차총승객수,등록일자
0,20200601,9709,9709번(파주시 맥금동~서울역),12023,불광역3호선.서울혁신파크,7,1,20200604
1,20200601,9709,9709번(파주시 맥금동~서울역),12026,한전성서지사.북한산푸르지오,5,5,20200604
2,20200601,9709,9709번(파주시 맥금동~서울역),12025,한전성서지사.북한산푸르지오,1,2,20200604
3,20200601,9709,9709번(파주시 맥금동~서울역),12028,녹번역,3,8,20200604
4,20200601,9709,9709번(파주시 맥금동~서울역),12027,녹번역,9,3,20200604
...,...,...,...,...,...,...,...,...
1162348,20200630,9714,9714번(교하운정~서울역),63657,트리플메디컬타운,90,1,20200703
1162349,20200630,9714,9714번(교하운정~서울역),63659,중앙공원,32,0,20200703
1162350,20200630,9714,9714번(교하운정~서울역),63674,책향기마을,38,1,20200703
1162351,20200630,9714,9714번(교하운정~서울역),36601,대화역,18,71,20200703


In [47]:
#파일 저장
df6.to_csv("BUS_STATION_BOARDING_MONTH_202006.csv")

In [49]:
#파일 확인
df6 = pd.read_csv("./BUS_STATION_BOARDING_MONTH_202006.csv", encoding="utf-8")
df6

,Unnamed: 0,사용일자,노선번호,노선명,버스정류장ARS번호,역명,승차총승객수,하차총승객수,등록일자
0,0,20200601,9709,9709번(파주시 맥금동~서울역),12023,불광역3호선.서울혁신파크,7,1,20200604
1,1,20200601,9709,9709번(파주시 맥금동~서울역),12026,한전성서지사.북한산푸르지오,5,5,20200604
2,2,20200601,9709,9709번(파주시 맥금동~서울역),12025,한전성서지사.북한산푸르지오,1,2,20200604
3,3,20200601,9709,9709번(파주시 맥금동~서울역),12028,녹번역,3,8,20200604
4,4,20200601,9709,9709번(파주시 맥금동~서울역),12027,녹번역,9,3,20200604
...,...,...,...,...,...,...,...,...,...
1162348,1162348,20200630,9714,9714번(교하운정~서울역),63657,트리플메디컬타운,90,1,20200703
1162349,1162349,20200630,9714,9714번(교하운정~서울역),63659,중앙공원,32,0,20200703
1162350,1162350,20200630,9714,9714번(교하운정~서울역),63674,책향기마을,38,1,20200703
1162351,1162351,20200630,9714,9714번(교하운정~서울역),36601,대화역,18,71,20200703


In [50]:
#필요없는 컬럼 삭제
df6 = df6.drop("Unnamed: 0", axis=1)
df6

,사용일자,노선번호,노선명,버스정류장ARS번호,역명,승차총승객수,하차총승객수,등록일자
0,20200601,9709,9709번(파주시 맥금동~서울역),12023,불광역3호선.서울혁신파크,7,1,20200604
1,20200601,9709,9709번(파주시 맥금동~서울역),12026,한전성서지사.북한산푸르지오,5,5,20200604
2,20200601,9709,9709번(파주시 맥금동~서울역),12025,한전성서지사.북한산푸르지오,1,2,20200604
3,20200601,9709,9709번(파주시 맥금동~서울역),12028,녹번역,3,8,20200604
4,20200601,9709,9709번(파주시 맥금동~서울역),12027,녹번역,9,3,20200604
...,...,...,...,...,...,...,...,...
1162348,20200630,9714,9714번(교하운정~서울역),63657,트리플메디컬타운,90,1,20200703
1162349,20200630,9714,9714번(교하운정~서울역),63659,중앙공원,32,0,20200703
1162350,20200630,9714,9714번(교하운정~서울역),63674,책향기마을,38,1,20200703
1162351,20200630,9714,9714번(교하운정~서울역),36601,대화역,18,71,20200703


In [51]:
#다시 csv파일로 저장
df6.to_csv("BUS_STATION_BOARDING_MONTH_202006_2.csv")

In [53]:
#컬럼명 오류인 파일 불러오기4
df5 = pd.read_csv("./data2/2020/BUS_STATION_BOARDING_MONTH_202005.txt", encoding="euc-kr", names = ["사용일자","지울것","노선번호","노선명","버스정류장ARS번호","역명","승차총승객수","하차총승객수","등록일자"], skiprows=[0])
df5

,사용일자,지울것,노선번호,노선명,버스정류장ARS번호,역명,승차총승객수,하차총승객수,등록일자
0,20200501,11110155,1154,1154번(하계동~장암동),11358,롯데마트,146,13,20200504
1,20200501,11110814,1167,1167번(우이동~은행사거리),10337,방학중학교,21,70,20200504
2,20200501,11110957,3316,3316번(마천동~천호역),24279,종점,31,0,20200504
3,20200501,91000034,9714,9714번(교하운정~서울역),12008,디지털미디어시티역,20,2,20200504
4,20200501,11110966,320,320번(송파차고지~상봉터미널),07313,동원시장,188,80,20200504
...,...,...,...,...,...,...,...,...,...
1199569,20200531,11110056,601,601번(개화동~종로4가),16013,발산역,90,322,20200603
1199570,20200531,41110065,6638,6638번(철산동~오목교),38075,철산우성아파트,0,34,20200603
1199571,20200531,41110065,6638,6638번(철산동~오목교),17166,경인중학교,152,60,20200603
1199572,20200531,41110065,6638,6638번(철산동~오목교),38062,세풍운수종점,12,1,20200603


In [54]:
#컬럼명 갖게 하기 위해 필요없는컬럼 제거
df5 = df5.drop(["지울것"],axis=1)
df5

,사용일자,노선번호,노선명,버스정류장ARS번호,역명,승차총승객수,하차총승객수,등록일자
0,20200501,1154,1154번(하계동~장암동),11358,롯데마트,146,13,20200504
1,20200501,1167,1167번(우이동~은행사거리),10337,방학중학교,21,70,20200504
2,20200501,3316,3316번(마천동~천호역),24279,종점,31,0,20200504
3,20200501,9714,9714번(교하운정~서울역),12008,디지털미디어시티역,20,2,20200504
4,20200501,320,320번(송파차고지~상봉터미널),07313,동원시장,188,80,20200504
...,...,...,...,...,...,...,...,...
1199569,20200531,601,601번(개화동~종로4가),16013,발산역,90,322,20200603
1199570,20200531,6638,6638번(철산동~오목교),38075,철산우성아파트,0,34,20200603
1199571,20200531,6638,6638번(철산동~오목교),17166,경인중학교,152,60,20200603
1199572,20200531,6638,6638번(철산동~오목교),38062,세풍운수종점,12,1,20200603


In [55]:
#파일 저장
df5.to_csv("BUS_STATION_BOARDING_MONTH_202005.csv")

In [56]:
#파일 확인
df5 = pd.read_csv("./BUS_STATION_BOARDING_MONTH_202005.csv", encoding="utf-8", names = ["사용일자","노선번호","노선명","버스정류장ARS번호","역명","승차총승객수","하차총승객수","등록일자"], skiprows=[0])
df5

,사용일자,노선번호,노선명,버스정류장ARS번호,역명,승차총승객수,하차총승객수,등록일자
0,20200501,1154,1154번(하계동~장암동),11358,롯데마트,146,13,20200504
1,20200501,1167,1167번(우이동~은행사거리),10337,방학중학교,21,70,20200504
2,20200501,3316,3316번(마천동~천호역),24279,종점,31,0,20200504
3,20200501,9714,9714번(교하운정~서울역),12008,디지털미디어시티역,20,2,20200504
4,20200501,320,320번(송파차고지~상봉터미널),07313,동원시장,188,80,20200504
...,...,...,...,...,...,...,...,...
1199569,20200531,601,601번(개화동~종로4가),16013,발산역,90,322,20200603
1199570,20200531,6638,6638번(철산동~오목교),38075,철산우성아파트,0,34,20200603
1199571,20200531,6638,6638번(철산동~오목교),17166,경인중학교,152,60,20200603
1199572,20200531,6638,6638번(철산동~오목교),38062,세풍운수종점,12,1,20200603


In [ ]:
df5 = pd.read_csv("./BUS_STATION_BOARDING_MONTH_202005.csv", encoding="utf-8", names = ["사용일자","노선번호","노선명","버스정류장ARS번호","역명","승차총승객수","하차총승객수","등록일자"], skiprows=[0])
df5

### txt 변환 완료!

## 2. merge 과정

* 연도별로 나눠서 merge
* 용량이 너무 커서 parquet으로 저장하려고함

### 2021년

In [6]:
#csv 합치기
input_file = r"C:/Users/chahn/Documents/이어드림/14주차 중간프로젝트/버스/data2/2021"
output_file = r'C:/Users/chahn/Documents/이어드림/14주차 중간프로젝트/버스/2021_bus_per_date.txt'

allFile_list = glob.glob(os.path.join(input_file,'BUS_STATION_BOARDING_MONTH_*'))
print(allFile_list)
allData = []
for file in allFile_list:
    df = pd.read_csv(file, encoding='euc-kr', names = ["사용일자","노선번호","노선명","버스정류장ARS번호","역명","승차총승객수","하차총승객수","등록일자"], skiprows=[0])
    allData.append(df)

dataCombine = pd.concat(allData, axis=0, ignore_index=True)
dataCombine.to_csv(output_file,encoding='euc-kr', index=False)

['C:/Users/chahn/Documents/이어드림/14주차 중간프로젝트/버스/data2/2021\\BUS_STATION_BOARDING_MONTH_202101.txt', 'C:/Users/chahn/Documents/이어드림/14주차 중간프로젝트/버스/data2/2021\\BUS_STATION_BOARDING_MONTH_202102.txt', 'C:/Users/chahn/Documents/이어드림/14주차 중간프로젝트/버스/data2/2021\\BUS_STATION_BOARDING_MONTH_202103.txt', 'C:/Users/chahn/Documents/이어드림/14주차 중간프로젝트/버스/data2/2021\\BUS_STATION_BOARDING_MONTH_202104.txt', 'C:/Users/chahn/Documents/이어드림/14주차 중간프로젝트/버스/data2/2021\\BUS_STATION_BOARDING_MONTH_202105.txt', 'C:/Users/chahn/Documents/이어드림/14주차 중간프로젝트/버스/data2/2021\\BUS_STATION_BOARDING_MONTH_202106.txt', 'C:/Users/chahn/Documents/이어드림/14주차 중간프로젝트/버스/data2/2021\\BUS_STATION_BOARDING_MONTH_202107.txt', 'C:/Users/chahn/Documents/이어드림/14주차 중간프로젝트/버스/data2/2021\\BUS_STATION_BOARDING_MONTH_202108.txt', 'C:/Users/chahn/Documents/이어드림/14주차 중간프로젝트/버스/data2/2021\\BUS_STATION_BOARDING_MONTH_202109.txt', 'C:/Users/chahn/Documents/이어드림/14주차 중간프로젝트/버스/data2/2021\\BUS_STATION_BOARDING_MONTH_202110.txt']


C:\Users\chahn\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3331: DtypeWarning: Columns (6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\chahn\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3331: DtypeWarning: Columns (0,5,7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\chahn\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3331: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\chahn\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3331: DtypeWarning: Columns (0,5,6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\chahn\anaconda3\lib\site-packages\IPython\core\in

In [8]:
#확인
df1 = pd.read_csv("./2021_bus_per_date.txt", encoding="euc-kr", names = ["사용일자","노선번호","노선명","버스정류장ARS번호","역명","승차총승객수","하차총승객수","등록일자"], skiprows=[0])
df1

C:\Users\chahn\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3331: DtypeWarning: Columns (0,5,6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,사용일자,노선번호,노선명,버스정류장ARS번호,역명,승차총승객수,하차총승객수,등록일자
0,20210101,100,100번(하계동~용산구청),11428,한성여객종점,12.0,0,20210104
1,20210101,100,100번(하계동~용산구청),11374,서울시립북서울미술관,8.0,0,20210104
2,20210101,100,100번(하계동~용산구청),11380,목련아파트상가,86.0,14,20210104
3,20210101,100,100번(하계동~용산구청),11339,월계헬스케어센터,112.0,44,20210104
4,20210101,100,100번(하계동~용산구청),01002,창경궁.서울대학교병원,21.0,23,20210104
...,...,...,...,...,...,...,...,...
11673774,20211031,마포11,마포11(신촌전철역~마포역),14856,굴다리시장,33,21,20211103.0
11673775,20211031,5714,5714번(광명공영차고지~이대입구),19120,영등포시장,233,346,20211103.0
11673776,20211031,마포12,마포12(신촌전철역~마포역),14855,신석초등학교,70,66,20211103.0
11673777,20211031,173,173번(월계동~연세대),14152,서울대동창회관,76,67,20211103.0


In [9]:
#인덱스 값 확인
df1["사용일자"].unique()

array([20210101, 20210102, 20210103, 20210104, 20210105, 20210211,
       20210106, 20210107, 20210108, 20210109, 20210110, 20210111,
       20210112, 20210113, 20210114, 20210115, 20210116, 20210117,
       20210118, 20210119, 20210120, 20210121, 20210122, 20210123,
       20210124, 20210125, 20210126, 20210127, 20210128, 20210129,
       20210130, 20210131, 20210201, 20210202, 20210203, 20210204,
       20210205, 20210206, 20210207, 20210208, 20210209, 20210210,
       20210427, 20210212, '20210212', '20210213', '202악攬英맏뮐側?,29"',
       '20210829', '20210214', 20210214, 20210215, 20210216, 20210217,
       20210218, 20210219, 20210220, 20210221, 20210222, 20210223,
       20210224, 20210225, 20210226, 20210227, 20210228, 20210301,
       20210302, 20210303, 20210304, 20210305, 20210821, 20210306,
       20210307, 20210308, 20210309, 20210310, 20210311, 314, 20210312,
       20210220210312, 20210313, 20210314, 20210315, 20210316, 20210317,
       20210426, 20210318, 20210319, 20210320

### 2020년

In [8]:
#csv 합치기
input_file = r"C:/Users/chahn/Documents/이어드림/14주차 중간프로젝트/버스/data2/2020"
output_file = r'C:/Users/chahn/Documents/이어드림/14주차 중간프로젝트/버스/2020_bus_per_date.txt'

allFile_list = glob.glob(os.path.join(input_file,'BUS_STATION_BOARDING_MONTH_*'))
print(allFile_list)
allData = []
for file in allFile_list:
    df = pd.read_csv(file, encoding='euc-kr', names = ["사용일자","노선번호","노선명","버스정류장ARS번호","역명","승차총승객수","하차총승객수","등록일자"], skiprows=[0])
    allData.append(df)

dataCombine = pd.concat(allData, axis=0, ignore_index=True)
dataCombine.to_csv(output_file,encoding='euc-kr', index=False)

['C:/Users/chahn/Documents/이어드림/14주차 중간프로젝트/버스/data2/2020\\BUS_STATION_BOARDING_MONTH_202001.txt', 'C:/Users/chahn/Documents/이어드림/14주차 중간프로젝트/버스/data2/2020\\BUS_STATION_BOARDING_MONTH_202002.txt', 'C:/Users/chahn/Documents/이어드림/14주차 중간프로젝트/버스/data2/2020\\BUS_STATION_BOARDING_MONTH_202003.txt', 'C:/Users/chahn/Documents/이어드림/14주차 중간프로젝트/버스/data2/2020\\BUS_STATION_BOARDING_MONTH_202004.txt', 'C:/Users/chahn/Documents/이어드림/14주차 중간프로젝트/버스/data2/2020\\BUS_STATION_BOARDING_MONTH_202005.txt', 'C:/Users/chahn/Documents/이어드림/14주차 중간프로젝트/버스/data2/2020\\BUS_STATION_BOARDING_MONTH_202006.txt', 'C:/Users/chahn/Documents/이어드림/14주차 중간프로젝트/버스/data2/2020\\BUS_STATION_BOARDING_MONTH_202007.txt', 'C:/Users/chahn/Documents/이어드림/14주차 중간프로젝트/버스/data2/2020\\BUS_STATION_BOARDING_MONTH_202008.txt', 'C:/Users/chahn/Documents/이어드림/14주차 중간프로젝트/버스/data2/2020\\BUS_STATION_BOARDING_MONTH_202009.txt', 'C:/Users/chahn/Documents/이어드림/14주차 중간프로젝트/버스/data2/2020\\BUS_STATION_BOARDING_MONTH_202010.txt', 'C:/Users/chahn/Doc

C:\Users\chahn\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3331: DtypeWarning: Columns (5,6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\chahn\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3331: DtypeWarning: Columns (5,7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\chahn\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3331: DtypeWarning: Columns (6,7,8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\chahn\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3331: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\chahn\anaconda3\lib\site-packages\IPython\core\in

In [10]:
#확인
df2 = pd.read_csv("./2020_bus_per_date.txt", encoding="euc-kr", names = ["사용일자","노선번호","노선명","버스정류장ARS번호","역명","승차총승객수","하차총승객수","등록일자"], skiprows=[0])
df2

C:\Users\chahn\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3331: DtypeWarning: Columns (0,5,6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,사용일자,노선번호,노선명,버스정류장ARS번호,역명,승차총승객수,하차총승객수,등록일자
0,20200101,N62,N62번(양천공영차고지~면목동차고지),04133,왕십리역,0,13,20200104
1,20200101,N62,N62번(양천공영차고지~면목동차고지),04136,행당1동주민센터.성동소방서,2,3,20200104
2,20200101,N62,N62번(양천공영차고지~면목동차고지),04186,성동교남단,0,2,20200104
3,20200101,N62,N62번(양천공영차고지~면목동차고지),04256,성수사거리,0,3,20200104
4,20200101,N62,N62번(양천공영차고지~면목동차고지),05115,국립서울병원앞,1,0,20200104
...,...,...,...,...,...,...,...,...
14125693,20201231,서초13,서초13(이수역~동덕여고),22900,동덕여고,116.0,168,20210103
14125694,20201231,5714,5714번(광명공영차고지~이대입구),19125,삼환아파트,241.0,229,20210103
14125695,20201231,마포01,마포01(산천동리버힐삼성아파트~용문시장),03520,도원삼성래미안아파트101동앞,5.0,5,20210103
14125696,20201231,8331,8331번(마천사거리~잠실역),24302,마천2동윤진빌딩.신동아아파트,44.0,0,20210103


In [11]:
#인덱스 값 확인
df2["사용일자"].unique()

array([20200101, 20200102, 20200103, 20200104, 20200105, 20200106,
       20200107, 20200108, 20200109, 20200110, 20200111, 20200112,
       20200113, 20200114, 20200115, 20200116, 20200117, 20200118,
       20200119, 20200120, 20200121, 20200122, 20200123, 20200124,
       20200125, 20200126, 20200127, 20200128, 20200129, 20200130,
       20200131, 20200201, 20200202, 20200203, 20200204, 20200205,
       20200206, 20200207, 20200208, 20200209, 20200210, 20200211,
       20200212, 20200213, 20200214, 20200215, 20200216, 20200217,
       20200218, 20200219, 20200220, 20200221, 20200222, 20200223,
       20200224, 20200225, 20200226, 20200227, 20200228, 20200229,
       20200301, 20200725, 20200302, 20200303, 20200304, 20200305,
       20200306, 20200307, 20200308, 20200309, 20200310, 20200311,
       20200312, 20200313, 20200314, 20200315, 20200316, 20200517,
       20200317, 20200318, 20200319, 20200320, 20200321, 20200322,
       20200323, 20200324, 20200325, 20200326, 20200327, 20200

### 2019년

In [2]:
#csv 합치기
input_file = r"C:/Users/chahn/Documents/이어드림/14주차 중간프로젝트/버스/data2/2019"
output_file = r'C:/Users/chahn/Documents/이어드림/14주차 중간프로젝트/버스/2019_bus_per_date.txt'

allFile_list = glob.glob(os.path.join(input_file,'BUS_STATION_BOARDING_MONTH_*'))
print(allFile_list)
allData = []
for file in allFile_list:
    df = pd.read_csv(file, encoding='euc-kr', names = ["사용일자","노선번호","노선명","버스정류장ARS번호","역명","승차총승객수","하차총승객수","등록일자"], skiprows=[0])
    allData.append(df)

dataCombine = pd.concat(allData, axis=0, ignore_index=True)
dataCombine.to_csv(output_file,encoding='euc-kr', index=False)

['C:/Users/chahn/Documents/이어드림/14주차 중간프로젝트/버스/data2/2019\\BUS_STATION_BOARDING_MONTH_201901.txt', 'C:/Users/chahn/Documents/이어드림/14주차 중간프로젝트/버스/data2/2019\\BUS_STATION_BOARDING_MONTH_201902.txt', 'C:/Users/chahn/Documents/이어드림/14주차 중간프로젝트/버스/data2/2019\\BUS_STATION_BOARDING_MONTH_201903.txt', 'C:/Users/chahn/Documents/이어드림/14주차 중간프로젝트/버스/data2/2019\\BUS_STATION_BOARDING_MONTH_201904.txt', 'C:/Users/chahn/Documents/이어드림/14주차 중간프로젝트/버스/data2/2019\\BUS_STATION_BOARDING_MONTH_201905.txt', 'C:/Users/chahn/Documents/이어드림/14주차 중간프로젝트/버스/data2/2019\\BUS_STATION_BOARDING_MONTH_201906.txt', 'C:/Users/chahn/Documents/이어드림/14주차 중간프로젝트/버스/data2/2019\\BUS_STATION_BOARDING_MONTH_201907.txt', 'C:/Users/chahn/Documents/이어드림/14주차 중간프로젝트/버스/data2/2019\\BUS_STATION_BOARDING_MONTH_201908.txt', 'C:/Users/chahn/Documents/이어드림/14주차 중간프로젝트/버스/data2/2019\\BUS_STATION_BOARDING_MONTH_201909.txt', 'C:/Users/chahn/Documents/이어드림/14주차 중간프로젝트/버스/data2/2019\\BUS_STATION_BOARDING_MONTH_201910.txt', 'C:/Users/chahn/Doc

C:\Users\chahn\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3331: DtypeWarning: Columns (5,6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\chahn\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3331: DtypeWarning: Columns (0,5,6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\chahn\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3331: DtypeWarning: Columns (5,6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
#확인
df3 = pd.read_csv("./2019_bus_per_date.txt", encoding="euc-kr", names = ["사용일자","노선번호","노선명","버스정류장ARS번호","역명","승차총승객수","하차총승객수","등록일자"], skiprows=[0])
df3

,사용일자,노선번호,노선명,버스정류장ARS번호,역명,승차총승객수,하차총승객수,등록일자
0,20190101,100,100번(하계동~용산구청),11428,한성여객종점,10.0,0.0,20190104.0
1,20190101,100,100번(하계동~용산구청),11387,노원평생학습관,5.0,0.0,20190104.0
2,20190101,100,100번(하계동~용산구청),11373,중계역,27.0,1.0,20190104.0
3,20190101,100,100번(하계동~용산구청),11381,중계목화아파트4단지,110.0,12.0,20190104.0
4,20190101,100,100번(하계동~용산구청),11311,인덕대학,87.0,76.0,20190104.0
...,...,...,...,...,...,...,...,...
14005301,20191231,영등포01,영등포01(신도림역~구로디지털단지역),19542,농협.대림2동주민센터,108,86,20200103
14005302,20191231,6640B,6640B번(양천차고지~양천차고지),17202,천왕역,171,101,20200103
14005303,20191231,2112,2112번(면목동~성북동),06268,장평중학교,42,41,20200103
14005304,20191231,은평10,은평10(숭실고등학교~산새마을),12160,상신초등학교,111,55,20200103


In [4]:
#인덱스 확인
df3["사용일자"].unique()

array([20190101, 20190102, 20190103, 20190324, 201903249, 20190104,
       20190105, 20190106, 20190107, 20190108, 20190109, 20190110,
       20190111, 20190112, 20190113, 20190114, 20190115, 20190116,
       20190117, 20190118, 20190119, 20190120, 20190121, 20190122,
       20190123, 20190124, 20190125, 20190126, 20190127, 20190128,
       20190129, 20190130, 20190131, 20190201, 20190202, 20190203,
       20190204, 20190205, 20190206, 20190207, 20190208, 20190306,
       20190209, 20190210, 20190211, 20190212, 20190213, 20190214,
       20190215, 20190216, 20190217, 20190218, 20190219, 20190719,
       20190220, 20190221, 20190222, 20190223, 20190224, 20190225,
       20190226, 20190723, 20190301, 20190227, 20190228, 20190302,
       20190303, '20190303', '20190204', '2019020,50"', '20190304',
       '20190305', 20190305, 20190307, 20190308, 20190309, 20190310,
       20190311, 20190716, 20190312, 20190313, 20190314, 20190315,
       20190316, 20190517, 20190317, '20190317', '20190318

### 2018년

In [5]:
#csv 합치기
input_file = r"C:/Users/chahn/Documents/이어드림/14주차 중간프로젝트/버스/data2/2018"
output_file = r'C:/Users/chahn/Documents/이어드림/14주차 중간프로젝트/버스/2018_bus_per_date.txt'

allFile_list = glob.glob(os.path.join(input_file,'BUS_STATION_BOARDING_MONTH_*'))
print(allFile_list)
allData = []
for file in allFile_list:
    df = pd.read_csv(file, encoding='euc-kr', names = ["사용일자","노선번호","노선명","버스정류장ARS번호","역명","승차총승객수","하차총승객수","등록일자"], skiprows=[0])
    allData.append(df)

dataCombine = pd.concat(allData, axis=0, ignore_index=True)
dataCombine.to_csv(output_file,encoding='euc-kr', index=False)

['C:/Users/chahn/Documents/이어드림/14주차 중간프로젝트/버스/data2/2018\\BUS_STATION_BOARDING_MONTH_201801.txt', 'C:/Users/chahn/Documents/이어드림/14주차 중간프로젝트/버스/data2/2018\\BUS_STATION_BOARDING_MONTH_201802.txt', 'C:/Users/chahn/Documents/이어드림/14주차 중간프로젝트/버스/data2/2018\\BUS_STATION_BOARDING_MONTH_201803.txt', 'C:/Users/chahn/Documents/이어드림/14주차 중간프로젝트/버스/data2/2018\\BUS_STATION_BOARDING_MONTH_201804.txt', 'C:/Users/chahn/Documents/이어드림/14주차 중간프로젝트/버스/data2/2018\\BUS_STATION_BOARDING_MONTH_201805.txt', 'C:/Users/chahn/Documents/이어드림/14주차 중간프로젝트/버스/data2/2018\\BUS_STATION_BOARDING_MONTH_201806.txt', 'C:/Users/chahn/Documents/이어드림/14주차 중간프로젝트/버스/data2/2018\\BUS_STATION_BOARDING_MONTH_201807.txt', 'C:/Users/chahn/Documents/이어드림/14주차 중간프로젝트/버스/data2/2018\\BUS_STATION_BOARDING_MONTH_201808.txt', 'C:/Users/chahn/Documents/이어드림/14주차 중간프로젝트/버스/data2/2018\\BUS_STATION_BOARDING_MONTH_201809.txt', 'C:/Users/chahn/Documents/이어드림/14주차 중간프로젝트/버스/data2/2018\\BUS_STATION_BOARDING_MONTH_201810.txt', 'C:/Users/chahn/Doc

In [6]:
#확인
df4 = pd.read_csv("./2018_bus_per_date.txt", encoding="euc-kr", names = ["사용일자","노선번호","노선명","버스정류장ARS번호","역명","승차총승객수","하차총승객수","등록일자"], skiprows=[0])
df4

,사용일자,노선번호,노선명,버스정류장ARS번호,역명,승차총승객수,하차총승객수,등록일자
0,20180101,100,100번(하계동~용산구청),11428,한성여객종점,12,4,20180104
1,20180101,100,100번(하계동~용산구청),11387,노원평생학습관,11,0,20180104
2,20180101,100,100번(하계동~용산구청),11373,중계역,15,1,20180104
3,20180101,100,100번(하계동~용산구청),11381,중계시영4단지,97,13,20180104
4,20180101,100,100번(하계동~용산구청),11311,인덕대학,93,79,20180104
...,...,...,...,...,...,...,...,...
13748820,20181231,1144,1144번(하계동~삼양사거리),09102,우이동도선사입구.북한산우이역,19,171,20190103
13748821,20181231,1144,1144번(하계동~삼양사거리),09103,우이동성원아파트,7,37,20190103
13748822,20181231,1144,1144번(하계동~삼양사거리),09108,국립4.19민주묘지입구사거리,35,100,20190103
13748823,20181231,1144,1144번(하계동~삼양사거리),09106,서라벌중학교.우이동대우아파트,54,58,20190103


In [7]:
#인덱스 값 확인
df4["사용일자"].unique()

array([20180101, 20180102, 20180103, 20180104, 20180105, 20180106,
       20180107, 20180108, 20180109, 20180110, 20180111, 20180112,
       20180113, 20180114, 20180115, 20180116, 20180117, 20180118,
       20180119, 20180120, 20180121, 20180122, 20180123, 20180124,
       20180125, 20180126, 20180127, 20180128, 20180129, 20180130,
       20180131, 20180201, 20180202, 20180203, 20180204, 20180205,
       20180206, 20180207, 20180208, 20180209, 20180210, 20180211,
       20180212, 20180213, 20180214, 20180215, 20180216, 20180217,
       20180218, 20180219, 20180220, 20180221, 20180222, 20180223,
       20180224, 20180225, 20180226, 20180227, 20180228, 20180301,
       20180302, 20180303, 20180304, 20180305, 20180306, 20180307,
       20180308, 20180309, 20180310, 20180311, 20180312, 20180313,
       20180314, 20180315, 20180316, 20180317, 20180318, 20180319,
       20180320, 20180321, 20180322, 20180323, 20180324, 20180325,
       20180326, 20180327, 20180328, 20180329, 20180330, 20180

### parquet으로 변환!

In [13]:
df4.to_parquet('2018_bus_per_date.parquet')

In [4]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [71]:
#parquet 확인
pq = spark.read.parquet('./data/parquet/2018_bus_per_date.parquet')
pq.show()

+--------+--------+------------------------------+-----------------+----------------------+------------+------------+--------+
|사용일자|노선번호|                        노선명|버스정류장ARS번호|                  역명|승차총승객수|하차총승객수|등록일자|
+--------+--------+------------------------------+-----------------+----------------------+------------+------------+--------+
|20180101|     100|        100번(하계동~용산구청)|            11428|          한성여객종점|          12|           4|20180104|
|20180101|     100|        100번(하계동~용산구청)|            11387|        노원평생학습관|          11|           0|20180104|
|20180101|     100|        100번(하계동~용산구청)|            11373|                중계역|          15|           1|20180104|
|20180101|     100|        100번(하계동~용산구청)|            11381|         중계시영4단지|          97|          13|20180104|
|20180101|     100|        100번(하계동~용산구청)|            11311|              인덕대학|          93|          79|20180104|
|20180101|     100|        100번(하계동~용산구청)|            01198|                원남동|      

In [72]:
#parquet 확인
pq.select("사용일자").distinct().collect()

[Row(사용일자=20180812),
 Row(사용일자=20180927),
 Row(사용일자=20181115),
 Row(사용일자=20180105),
 Row(사용일자=20180425),
 Row(사용일자=20180515),
 Row(사용일자=20180601),
 Row(사용일자=20181003),
 Row(사용일자=20180208),
 Row(사용일자=20180523),
 Row(사용일자=20180809),
 Row(사용일자=20180423),
 Row(사용일자=20180401),
 Row(사용일자=20181125),
 Row(사용일자=20180513),
 Row(사용일자=20180806),
 Row(사용일자=20180122),
 Row(사용일자=20180316),
 Row(사용일자=20180822),
 Row(사용일자=20180219),
 Row(사용일자=20180302),
 Row(사용일자=20180501),
 Row(사용일자=20181126),
 Row(사용일자=20180402),
 Row(사용일자=20180127),
 Row(사용일자=20180404),
 Row(사용일자=20180419),
 Row(사용일자=20180420),
 Row(사용일자=20180719),
 Row(사용일자=20181021),
 Row(사용일자=20181104),
 Row(사용일자=20180905),
 Row(사용일자=20180512),
 Row(사용일자=20181105),
 Row(사용일자=20180123),
 Row(사용일자=20181108),
 Row(사용일자=20180913),
 Row(사용일자=20181024),
 Row(사용일자=20181112),
 Row(사용일자=20180916),
 Row(사용일자=20180214),
 Row(사용일자=20180618),
 Row(사용일자=20180817),
 Row(사용일자=20180818),
 Row(사용일자=20180330),
 Row(사용일자=20180416),
 Row(사용일자=20180917),
 Row(사용일자=201